In [1]:
from crewai import Crew, Agent, Task, LLM
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool
from dotenv import load_dotenv
import os
from pydantic import BaseModel
load_dotenv()

/Users/bahacelik/Library/Caches/pypoetry/virtualenvs/crewaitesting-Zytas3Xz-py3.13/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


True

In [ ]:
gemini_llm = LLM(
    model='gemini/gemini-2.5-flash-lite',
    temperature=0.5,
    api_key=os.getenv('GOOGLE_API_KEY'),
    timeout=3
)

#with another api key from advokatas account
gemini_llm_2 = LLM(
    model='gemini/gemini-2.5-flash-lite',
    temperature=0.5,
    api_key=os.getenv('GOOGLE_API_KEY_2'),
    timeout=3
)

# grok_llm = LLM(
#     model='xai/grok-4-0709',
#     temperature=0.5,
#     api_key=os.getenv('GROK_API_KEY'),
#     timeout=3
# )

cohere_llm = LLM(
    model='command-r',
    temperature=0.5,
    api_key=os.getenv('COHERE_API_KEY')
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [3]:
#analyst agent that goes through the market and selects stocks
financial_analyst = Agent(
    role='Senior Financial Data Analyst',
    goal='Monitor all the stock market to find stocks that are worth investigating further',
    backstory='''
            You are a CFA certified financial analyst specialised in stock market. This agent uses statistical modeling and
            machine learning to pick stocks that are worth investigating further and potentially investing. With years of experience
            in investing this agent beats the market with its cherry picked stocks that outperforms the market.
            ''',
    verbose=True,
    allow_delegation=True,
    tools=[search_tool,scrape_tool],
    llm=cohere_llm,
    max_retry_limit=3
)

#news follower agent that searches the news for that stock to gather more info
news_follower = Agent(
    role='Senior Industry Analyzer',
    goal='Given the company stocks, thoroughly analyze the industry and the company itself by following the news and trends that might affect the company',
    backstory='''
            You are a Industry Analyzer expert who is chronically online following all the news that might affect 
            the stocks that was given to you. You always use multiple news outlets that are respectable and correlate the news into
            stock value information.
    ''',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool,scrape_tool],
    llm=gemini_llm,
    max_retry_limit=3
)

#risk assessor agent that 
risk_assesor = Agent(
    role='Senior Risk Assesor',
    goal='Follow what is going on around the financial markets and the world in general that might risk our investments',
    backstory='''
        You are a senior risk assesor that has years of industry experience with big financial events that shapes the financial markets.
        you provide critical risk assesments for stockmarket that is crucial for decision making.
''',
    verbose=True,
    allow_delegation=True,
    tools=[search_tool,scrape_tool],
    llm=gemini_llm_2,
    max_retry_limit=3
)

#decision maker agent that makes the final decision regarding which stock to buy and how many. Also explains its reasoning
decision_maker = Agent(
    role='Decision Maker',
    goal='''
    Final decision maker who analyzes all the information at hand and makes decision regarding which stock to buy and how much
    based on our budget of {budget} euros.
    ''',
    backstory='''
        You are a investment fund manager that makes the final decision in purchasing which stocks and how much of each stock
        based on the analysis and findings of your employees. You are not afraid of taking risks as long as they are feasible by the
        research your team has done. Since you are highly educated and experienced decision maker you do not gamble, you make informed
        decisions.
    ''',
    verbose=True,
    allow_delegation=True,
    llm=gemini_llm_2,
    max_retry_limit=3
)

In [4]:
class StockDetails(BaseModel):
    name: str
    value: float
    reason: str
    newsAttached: str
    risk: str
    projectedIncrasePercentage: float
    timelineInMonths: int
    buy:bool
    amount: float

In [5]:
#analyst task
analyst_task= Task(
    description='''
                Continously monitor and analyze stock market data for selecting stocks that would give the highest profits
                with the lowest risk. If you are not sure you can make a risk analysis by Senior Risk Assesor or figure out what is
                going on in the industry by Senior Industry Analyzer agents. Select at least 10 stocks that are worth diving deeper.
                ''',
    expected_output='''
            All the details about the stock that are defined to understand the stock
            [name, value, reason]
            ''',
    agent=financial_analyst
)

#news follower task
news_follower_task= Task(
    description='''
                Analyze the global political and economical situation with the breaking and currrent news to give a broader perspective
                on the current stock market. The tension around the world, policy changes, anything that could be relevant for the given
                stocks is worth keeping in mind. Be up-to-date and use your expert historical and political experience to help us make more
                informed decisions.
                ''',
    expected_output='''
                    Stock name and the world events/news that happen and might happen that would have potential affects to each stock given
                    in the newsAttached slot 
                    ''',
    agent=news_follower
)

#risk assesor task
risk_assesor_task = Task(
    description='''
                Given the stocks, and news of each stock. Make a risk assesmont to that specific stock based on the given information about it.
                ''',
    expected_output='Risk assesment of that stock on a scale of 1-10 10 being the most risk and 1 being the least. in risk slot.',
    agent=risk_assesor
)

#decision maker task
decision_maker_task = Task(
    description='''
                Given all the information, make a prediction on how each stock will turn out using statistical measures and machine learning
                along with combining your experience. Make sure you take into account everything from the news of that given stock to risk
                assesment. If a given field is not enough for you to make a decision, invoke the specific agent responsible to get more
                information you need. Make sure the amount of the stocks you decided to buy is exactly this amount: {budget}
                ''',
    expected_output='The final decision on how the stock will change. Populate the [projectedIncreasePercentage, timeline, buy, amount fields] ensuring your investment portfolio is exactly this amount: {budget}',
    output_json=StockDetails,
    output_file='stock_market_report.json',
    agent=decision_maker
)

In [6]:
stock_market_crew = Crew(agents=[financial_analyst,news_follower,risk_assesor,decision_maker],
                         tasks=[analyst_task, news_follower_task, risk_assesor_task, decision_maker_task],
                         verbose=True)

stock_market_crew.kickoff(inputs={'budget': 1000})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8c0ceb4e-79d3-403d-8d7b-dffc2c094e24                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Data Analyst                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│                  Continously monitor and analyze stock market data for selecting stocks that would give the     │
│  highest profits                                                                                                │
│                  with the lowest risk. If you are not sure you can make a risk analysis by Senior Risk Assesor  │
│  or figure out what is                                                                                          │
│                  going on in the industry by Senior Industry Analyzer agents. Select at least 10 stocks that    │
│  are worth diving deeper.                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Financial Data Analyst                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Selected Stocks with Details**                                                                                 │
│  1. **Apple Inc. (AAPL)**                                                                                       │
│     - **Value**: $180 per share                                                                                 │
│     - **Reason**: Strong fundamentals, dominant market position in tech, consistent revenue growth, and low     │
│  beta (0.85).                                                                                                   │
│                                                                                                                 │
│  2. **Johnson & Johnson (JNJ)**                                                                                 │
│     - **Value**: $165 per share                                                                                 │
│     - **Reason**: Defensive healthcare stock with stable dividends, low volatility, and resilient business      │
│  model.                                                                                                         │
│                                                                                                                 │
│  3. **NVIDIA Corporation (NVDA)**                                                                               │
│     - **Value**: $450 per share                                                                                 │
│     - **Reason**: Leader in AI and semiconductor industry, high growth potential, but higher risk due to        │
│  valuation.                                                                                                     │
│                                                                                                                 │
│  4. **Procter & Gamble (PG)**                                                                                   │
│     - **Value**: $145 per share                                                                                 │
│     - **Reason**: Consumer staples company with consistent performance, low risk, and strong brand portfolio.   │
│                                                                                                                 │
│  5. **NextEra Energy (NEE)**                                                                                    │
│     - **Value**: $80 per share                                                                                  │
│     - **Reason**: Renewable energy leader, government policy support, and stable dividend growth.               │
│                                                                                                                 │
│  6. **Microsoft Corporation (MSFT)**                                                                            │
│     - **Value**: $320 per share                                                                                 │
│     - **Reason**: Diversified tech giant with strong cloud business, low beta (0.90), and consistent            │
│  profitability.                                                                                                 │
│                                                                                                                 │
│  7. **Visa Inc. (V)**                                  

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 48121076-0db6-47cc-909e-fbd4b7d735a3                                                                     │
│  Agent: Senior Financial Data Analyst                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Industry Analyzer                                                                                │
│                                                                                                                 │
│  Task:                                                                                                          │
│                  Analyze the global political and economical situation with the breaking and currrent news to   │
│  give a broader perspective                                                                                     │
│                  on the current stock market. The tension around the world, policy changes, anything that       │
│  could be relevant for the given                                                                                │
│                  stocks is worth keeping in mind. Be up-to-date and use your expert historical and political    │
│  experience to help us make more                                                                                │
│                  informed decisions.                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'impact of global economic and political situation on renewable energy digital      │
│  payments home improvement waste management stocks', 'type': 'search', 'num': 10, 'engine': 'google'},          │
│  'organic': [{'title': 'The impact of digital economy on renewable energy development in ...', 'link':          │
│  'https://www.sciencedirect.com/science/article/pii/S2949753123000620', 'snippet': 'Missing: payments waste',   │
│  'position': 1}, {'title': 'A global perspective on the nexus between energy and stock ...', 'link':            │
│  'https://www.sciencedirect.com/science/article/abs/pii/S0140988324001142', 'snippet': 'This paper revisits     │
│  the relationship between energy and stock markets by accounting for renewable energy through the use of the    │
│  Divisia index method.', 'position': 2}, {'title': 'Does the Digital Economy Promote Renewable                  │
│  Energy?—Evidence ...', 'link': 'https://onlinelibrary.wiley.com/doi/10.1002/app5.70015', 'snippet': 'Missing:  │
│  payments waste', 'position': 3}, {'title': '2025 Renewable Energy Industry Outlook | Deloitte Insights',       │
│  'link':                                                                                                        │
│  'https://www.deloitte.com/us/en/insights/industry/renewable-energy/renewable-energy-industry-outlook.html',    │
│  'snippet': 'Missing: payments waste', 'position': 4}, {'title': 'Impacts of COVID-19 pandemic on the global    │
│  energy system and the ...', 'link': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC8455103/', 'snippet': 'Missing:  │
│  digital waste', 'position': 5}, {'title': '2025 Power and Utilities Industry Outlook | Deloitte Insights',     │
│  'link':                                                                                                        │
│  'https://www.deloitte.com/us/en/insights/industry/power-and-utilities/power-and-utilities-industry-outlook.ht  │
│  ml', 'snippet': 'Utilities are navigating a new era of growth and transformation as they address emerging      │
│  challenges and rising demand.', 'position': 6}, {'title': "How Sheinbaum's Energy Policies Could Reshape       │
│  Mexico's ...", 'link':                                                                                         │
│  'https://www.bakerinstitute.org/research/how-sheinbaums-energy-policies-could-reshape-mexicos-electricity-sec  │
│  tor', 'snippet': "For Mexico, limiting the installati...                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.Timeout: Connection timed out after None seconds.                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: a7c56421-7904-4666-b296-ec335b905e16                                                                     │
│  Agent: Senior Industry Analyzer                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 8c0ceb4e-79d3-403d-8d7b-dffc2c094e24                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Timeout: litellm.Timeout: Connection timed out after None seconds.